In [1]:
# dependencies
import pandas as pd

In [141]:
# read excel files into dataframes

season = "2017_2018"

file_to_load = f"Resources/{season}.xlsx"
df = pd.read_excel(file_to_load)

df.head()

,Player,FG,FGA,FG%,3P,FT,FTA,FT%,TRB,AST,STL,BLK,TOV,PS/G,VORP
0,Alex Abrines\abrinal01,1.5,3.9,0.395,1.1,0.5,0.6,0.848,1.5,0.4,0.5,0.1,0.3,4.7,-0.1
1,Quincy Acy\acyqu01,1.9,5.2,0.356,1.5,0.7,0.9,0.817,3.7,0.8,0.5,0.4,0.9,5.9,-0.1
2,Steven Adams\adamsst01,5.9,9.4,0.629,0.0,2.1,3.8,0.559,9.0,1.2,1.2,1.0,1.7,13.9,3.3
3,Bam Adebayo\adebaba01,2.5,4.9,0.512,0.0,1.9,2.6,0.721,5.5,1.5,0.5,0.6,1.0,6.9,0.8
4,Arron Afflalo\afflaar01,1.2,3.1,0.401,0.5,0.4,0.5,0.846,1.2,0.6,0.1,0.2,0.4,3.4,-0.7


In [142]:
# clean player names
df["Player"] = df["Player"].str.split("\\", n = 1, expand = True)
df.head()

,Player,FG,FGA,FG%,3P,FT,FTA,FT%,TRB,AST,STL,BLK,TOV,PS/G,VORP
0,Alex Abrines,1.5,3.9,0.395,1.1,0.5,0.6,0.848,1.5,0.4,0.5,0.1,0.3,4.7,-0.1
1,Quincy Acy,1.9,5.2,0.356,1.5,0.7,0.9,0.817,3.7,0.8,0.5,0.4,0.9,5.9,-0.1
2,Steven Adams,5.9,9.4,0.629,0.0,2.1,3.8,0.559,9.0,1.2,1.2,1.0,1.7,13.9,3.3
3,Bam Adebayo,2.5,4.9,0.512,0.0,1.9,2.6,0.721,5.5,1.5,0.5,0.6,1.0,6.9,0.8
4,Arron Afflalo,1.2,3.1,0.401,0.5,0.4,0.5,0.846,1.2,0.6,0.1,0.2,0.4,3.4,-0.7


In [143]:
# keep only total season stats for players with more than 1 team
df = df.drop_duplicates(subset=["Player"], keep = 'first')
df.head()

,Player,FG,FGA,FG%,3P,FT,FTA,FT%,TRB,AST,STL,BLK,TOV,PS/G,VORP
0,Alex Abrines,1.5,3.9,0.395,1.1,0.5,0.6,0.848,1.5,0.4,0.5,0.1,0.3,4.7,-0.1
1,Quincy Acy,1.9,5.2,0.356,1.5,0.7,0.9,0.817,3.7,0.8,0.5,0.4,0.9,5.9,-0.1
2,Steven Adams,5.9,9.4,0.629,0.0,2.1,3.8,0.559,9.0,1.2,1.2,1.0,1.7,13.9,3.3
3,Bam Adebayo,2.5,4.9,0.512,0.0,1.9,2.6,0.721,5.5,1.5,0.5,0.6,1.0,6.9,0.8
4,Arron Afflalo,1.2,3.1,0.401,0.5,0.4,0.5,0.846,1.2,0.6,0.1,0.2,0.4,3.4,-0.7


In [144]:
# enter roster size and number of teams in fantasy league
roster_size = 13
num_teams = 10
sample_size = roster_size*num_teams

In [145]:
top_players = df.sort_values("VORP", ascending = False).head(sample_size)
top_players["TOP"] = 1
top_players.head()

,Player,FG,FGA,FG%,3P,FT,FTA,FT%,TRB,AST,STL,BLK,TOV,PS/G,VORP,TOP
303,LeBron James,10.5,19.3,0.542,1.8,4.7,6.5,0.731,8.6,9.1,1.4,0.9,4.2,27.5,8.9,1
232,James Harden,9.0,20.1,0.449,3.7,8.7,10.1,0.858,5.4,8.8,1.8,0.7,4.4,30.4,8.3,1
625,Russell Westbrook,9.5,21.1,0.449,1.2,5.2,7.1,0.737,10.1,10.3,1.8,0.3,4.8,25.4,7.5,1
369,Damian Lillard,8.5,19.4,0.439,3.1,6.8,7.4,0.916,4.5,6.6,1.1,0.4,2.8,26.9,5.9,1
324,Nikola Jokic,6.7,13.5,0.499,1.5,3.5,4.2,0.850,10.7,6.1,1.2,0.8,2.8,18.5,5.6,1


In [146]:
df = pd.merge(top_players[["Player", "TOP"]], df, on="Player", how="outer").fillna(0)
df.head()

,Player,TOP,FG,FGA,FG%,3P,FT,FTA,FT%,TRB,AST,STL,BLK,TOV,PS/G,VORP
0,LeBron James,1.0,10.5,19.3,0.542,1.8,4.7,6.5,0.731,8.6,9.1,1.4,0.9,4.2,27.5,8.9
1,James Harden,1.0,9.0,20.1,0.449,3.7,8.7,10.1,0.858,5.4,8.8,1.8,0.7,4.4,30.4,8.3
2,Russell Westbrook,1.0,9.5,21.1,0.449,1.2,5.2,7.1,0.737,10.1,10.3,1.8,0.3,4.8,25.4,7.5
3,Damian Lillard,1.0,8.5,19.4,0.439,3.1,6.8,7.4,0.916,4.5,6.6,1.1,0.4,2.8,26.9,5.9
4,Nikola Jokic,1.0,6.7,13.5,0.499,1.5,3.5,4.2,0.850,10.7,6.1,1.2,0.8,2.8,18.5,5.6


In [147]:
# calculate adjusted percentages
top_players['adj_FG'] = (10*(top_players['FG'].sum() / sample_size) + top_players["FG"]) / (10*(top_players['FGA'].sum() / sample_size) + top_players["FGA"])
top_players['adj_FT'] = (10*(top_players['FT'].sum() / sample_size) + top_players["FT"]) / (10*(top_players['FTA'].sum() / sample_size) + top_players["FTA"])
top_players.head()

,Player,FG,FGA,FG%,3P,FT,FTA,FT%,TRB,AST,STL,BLK,TOV,PS/G,VORP,TOP,adj_FG,adj_FT
303,LeBron James,10.5,19.3,0.542,1.8,4.7,6.5,0.731,8.6,9.1,1.4,0.9,4.2,27.5,8.9,1,0.488576,0.767404
232,James Harden,9.0,20.1,0.449,3.7,8.7,10.1,0.858,5.4,8.8,1.8,0.7,4.4,30.4,8.3,1,0.474078,0.797220
625,Russell Westbrook,9.5,21.1,0.449,1.2,5.2,7.1,0.737,10.1,10.3,1.8,0.3,4.8,25.4,7.5,1,0.474276,0.768432
369,Damian Lillard,8.5,19.4,0.439,3.1,6.8,7.4,0.916,4.5,6.6,1.1,0.4,2.8,26.9,5.9,1,0.472782,0.803727
324,Nikola Jokic,6.7,13.5,0.499,1.5,3.5,4.2,0.850,10.7,6.1,1.2,0.8,2.8,18.5,5.6,1,0.480773,0.783276


In [148]:
df['adj_FG'] = (10*(top_players['FG'].sum() / sample_size) + df["FG"]) / (10*(top_players['FGA'].sum() / sample_size) + df["FGA"])
df['adj_FT'] = (10*(top_players['FT'].sum() / sample_size) + df["FT"]) / (10*(top_players['FTA'].sum() / sample_size) + df["FTA"])
df.head()

,Player,TOP,FG,FGA,FG%,3P,FT,FTA,FT%,TRB,AST,STL,BLK,TOV,PS/G,VORP,adj_FG,adj_FT
0,LeBron James,1.0,10.5,19.3,0.542,1.8,4.7,6.5,0.731,8.6,9.1,1.4,0.9,4.2,27.5,8.9,0.488576,0.767404
1,James Harden,1.0,9.0,20.1,0.449,3.7,8.7,10.1,0.858,5.4,8.8,1.8,0.7,4.4,30.4,8.3,0.474078,0.797220
2,Russell Westbrook,1.0,9.5,21.1,0.449,1.2,5.2,7.1,0.737,10.1,10.3,1.8,0.3,4.8,25.4,7.5,0.474276,0.768432
3,Damian Lillard,1.0,8.5,19.4,0.439,3.1,6.8,7.4,0.916,4.5,6.6,1.1,0.4,2.8,26.9,5.9,0.472782,0.803727
4,Nikola Jokic,1.0,6.7,13.5,0.499,1.5,3.5,4.2,0.850,10.7,6.1,1.2,0.8,2.8,18.5,5.6,0.480773,0.783276


In [149]:
# calculate z-scores and average z-score
df["zFG"] = (df["adj_FG"] - top_players["adj_FG"].mean()) / top_players["adj_FG"].std()
df["zFT"] = (df["adj_FT"] - top_players["adj_FT"].mean()) / top_players["adj_FT"].std()
df["z3P"] = (df["3P"] - top_players["3P"].mean()) / top_players["3P"].std()
df["zPTS"] = (df["PS/G"] - top_players["PS/G"].mean()) / top_players["PS/G"].std()
df["zREB"] = (df["TRB"] - top_players["TRB"].mean()) / top_players["TRB"].std()
df["zAST"] = (df["AST"] - top_players["AST"].mean()) / top_players["AST"].std()
df["zSTL"] = (df["STL"] - top_players["STL"].mean()) / top_players["STL"].std()
df["zBLK"] = (df["BLK"] - top_players["BLK"].mean()) / top_players["BLK"].std()
df["zTOV"] = (top_players["TOV"].mean() - df["TOV"]) / top_players["TOV"].std()
df["zAVG"] = (df["zFG"] + df["zFT"] + df["z3P"] + df["zPTS"] + df["zREB"] + df["zAST"] + df["zSTL"] + df["zBLK"] + df["zTOV"]) / 9
df.head()

,Player,TOP,FG,FGA,FG%,3P,FT,FTA,FT%,TRB,...,zFG,zFT,z3P,zPTS,zREB,zAST,zSTL,zBLK,zTOV,zAVG
0,LeBron James,1.0,10.5,19.3,0.542,1.8,4.7,6.5,0.731,8.6,...,1.988708,-0.897710,0.487241,2.211143,0.998435,2.751745,0.913735,0.426758,-2.640295,0.693307
1,James Harden,1.0,9.0,20.1,0.449,3.7,8.7,10.1,0.858,5.4,...,-0.997979,2.052433,2.431482,2.697889,-0.177525,2.612461,1.836340,0.017322,-2.859757,0.845852
2,Russell Westbrook,1.0,9.5,21.1,0.449,1.2,5.2,7.1,0.737,10.1,...,-0.957346,-0.796046,-0.126730,1.858672,1.549666,3.308879,1.836340,-0.801549,-3.298680,0.285912
3,Damian Lillard,1.0,8.5,19.4,0.439,3.1,6.8,7.4,0.916,4.5,...,-1.265004,2.696332,1.817511,2.110437,-0.508263,1.591048,0.221780,-0.596831,-1.104062,0.551439
4,Nikola Jokic,1.0,6.7,13.5,0.499,1.5,3.5,4.2,0.850,10.7,...,0.381117,0.672730,0.180256,0.700553,1.770159,1.358908,0.452432,0.222040,-1.104062,0.514904


In [150]:
# rank by avg z-score
df = df.sort_values("zAVG", ascending = False).reset_index(drop=True)

In [151]:
df.index += 1

In [152]:
df

,Player,TOP,FG,FGA,FG%,3P,FT,FTA,FT%,TRB,...,zFG,zFT,z3P,zPTS,zREB,zAST,zSTL,zBLK,zTOV,zAVG
1,Anthony Davis,1.0,10.4,19.5,0.534,0.7,6.6,8.0,0.828,11.1,...,1.674966,0.983011,-0.638372,2.311849,1.917154,-0.405351,1.144386,3.906961,-0.445676,1.160992
2,Stephen Curry,1.0,8.4,16.9,0.495,4.2,5.5,5.9,0.921,5.1,...,0.486972,2.445891,2.943124,2.026515,-0.287771,1.358908,1.375038,-1.006267,-1.323524,0.890987
3,Kevin Durant,1.0,9.3,18.0,0.516,2.5,5.3,5.9,0.889,6.8,...,1.081904,1.915347,1.203540,2.026515,0.336958,1.033913,-0.700826,2.269218,-1.323524,0.871450
4,James Harden,1.0,9.0,20.1,0.449,3.7,8.7,10.1,0.858,5.4,...,-0.997979,2.052433,2.431482,2.697889,-0.177525,2.612461,1.836340,0.017322,-2.859757,0.845852
5,Karl-Anthony Towns,1.0,7.8,14.3,0.545,1.5,4.2,4.9,0.858,12.3,...,1.563679,1.086252,0.180256,1.170514,2.358139,-0.358923,-0.470174,1.450347,-0.116484,0.762623
6,LeBron James,1.0,10.5,19.3,0.542,1.8,4.7,6.5,0.731,8.6,...,1.988708,-0.897710,0.487241,2.211143,0.998435,2.751745,0.913735,0.426758,-2.640295,0.693307
7,Giannis Antetokounmpo,1.0,9.9,18.7,0.529,0.6,6.5,8.5,0.760,10.0,...,1.498761,-0.240071,-0.740701,2.110437,1.512918,0.755346,1.144386,1.450347,-1.323524,0.685322
8,DeMarcus Cousins,1.0,8.5,18.0,0.470,2.2,6.1,8.2,0.746,12.9,...,-0.202406,-0.659307,0.896555,1.825103,2.578631,1.033913,1.375038,1.859783,-3.518142,0.576574
9,Chris Paul,1.0,6.3,13.8,0.460,2.5,3.5,3.8,0.919,5.4,...,-0.522150,1.553439,1.203540,0.717337,-0.177525,2.194610,1.605689,-1.006267,-0.445676,0.569222
10,Damian Lillard,1.0,8.5,19.4,0.439,3.1,6.8,7.4,0.916,4.5,...,-1.265004,2.696332,1.817511,2.110437,-0.508263,1.591048,0.221780,-0.596831,-1.104062,0.551439


In [153]:
df.to_excel(f"Resources/ranked_{season}.xlsx", index=True)